## Programming without indices (dot product)

### Preliminaries

<p><font color=red> Again, copy this notebook so that you don't corrupt the original!  Then you can "play" with the copy of the notebook all you want! </font> </p>

<p>In this notebook, we show how the FLAME notation (the notation in which vectors and/or matrices are partitioned into regions) can be leveraged to implement linear algebra operations without using indices (which are the root of all evil in programming...).</p>

Let's start by importing numpy and creating vectors $ x = \left( \begin{array}{r} 1 \\ 2 \\ 3 \end{array} \right) $ and $ y = \left( \begin{array}{r} -1 \\ 0 \\ -2 \end{array} \right) $.

Execute the code in the box by clicking in the box and then on "Cell -> Run".  Alternative, click on the box and push "Shift" and "Return" (or "Enter") together.

In [1]:
import numpy as np    # This imports a package called "numpy" that will make working with matrices 
                      # simpler

# create two two-dimensional matrices of only one column each.  
# In the future we will also think of (column) 
# vectors as matrices with only one column.
x = np.matrix( '1.;2.;3.' )
print( 'x = ' )
print( x )

y = np.matrix( '-1.;0.;-2.' )
print( 'y = ' )
print( y )

x = 
[[1.]
 [2.]
 [3.]]
y = 
[[-1.]
 [ 0.]
 [-2.]]


### Dot as a simple routine

<p>
Here is a simple routine for computing $ {\rm dot}( x, y ) = x^T y $:
</p>

In [2]:
def dot( x, y ):

    # Check how many elements there are in vector x.  For this, 
    # np.shape( x ) return the row and column size of x, where x is a matrix.
    
    m, n = np.shape( x )
    alpha = 0.0
    
    for i in range( m ):
        alpha = x[ i,0 ] * y[ i,0 ] + alpha
    
    return alpha

Be sure the run the above box, or this notebook won't know about the routine!!!

Now, execute

In [3]:
alpha = 0.

alpha = dot( x, y )

print( 'alpha' )
print( alpha )

print( 'Difference between  alpha and  np.transpose(x) * y:' )
alpha_reference = np.transpose(x) * y

print( alpha - alpha_reference[0,0]  )

alpha
-7.0
Difference between  alpha and  np.transpose(x) * y:
0.0


### An implementation with the FLAMEPy Application Programming Interface (API)

We now show how to implement this same routine using the FLAMEPy API.

Start by visiting the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a>.  Follow along with the video and paste the resulting code below.  Then follow along with the video and add the appropriate commands. (You may even want to bookmark this page).

Here is the algorithm as presented in Unit 1.6.2.  
<img src="https://studio.edge.edx.org/c4x/UTX/UT.5.01x/asset/FLAMEDot_alg.png" alt="some_text">

In the video for Unit 1.6.3, we discuss how to translate this into Python code using the FLAMEPy API.  Follow these instructions, insert the resulting code below.

In [4]:
# Insert Code here
import flame
import laff as laff

def dot_unb(x, y, alpha):

    xT, \                          # xT - top vec; xB - bottom vec;
    xB  = flame.part_2x1(x, \      # vec to be partitioned
                         0, 'TOP') # 0 elts in the TOP part

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    alpha[0, 0] = 0.0
    
    while xT.shape[0] < x.shape[0]:

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \        # x0 = xT
                                        xB, \        # [chi1; x2] = xB
                                        1, 'BOTTOM') # peels 1 elts off of bottom vec

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        alpha[0, 0] = chi1 * psi1 + alpha[0, 0]

        #------------------------------------------------------------#

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \ # [x0; chi1] = xT
                                         chi1, \ 
                                         x2,   \ # x2 = xB
                                         'TOP')  # add chi1 to top vec

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

In [5]:
import laff

alpha = np.matrix( '-2.0' ) # the way we are going to program, scalars, vectors, and matrices are 
                           # all just matrices.  So, alpha here is a 1 x 1 matrix, which we 
                           # initialize to some random number, in this case -2.0.

dot_unb( x, y, alpha )    # Takes x, y, and alpha as an input, and then updates alpha with the 
                          # result of dot( x, y ).  Notice that the contents of variable alpha
                          # are updated.  This only works if alpha is passed in as an array 
                          # (a matrix in our case)

print( 'alpha' )
print( alpha )

print( 'compare alpha to  np.transpose(x) * y:' )
alpha_reference = np.transpose(x) * y

print( alpha - alpha_reference[0,0]  )

alpha
[[-7.]]
compare alpha to  np.transpose(x) * y:
[[0.]]


The output should be:
<code>
alpha
[[-7.]]
compare alpha to  np.transpose(x) * y:
[[0.]]
</code>